In [1]:
"""
Extract gridMET variables 
Google Earth Engine (GEE) Python API
Author: maxwell.cook@colorado.edu
"""

import os, sys
import ee
import geemap
import time

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

ee.Authenticate()
ee.Initialize(project='jfsp-aspen')

print("Success !")

Success !


In [2]:
# Load gridded FRP data aggregated to first observation day
grid = ee.FeatureCollection('projects/jfsp-aspen/assets/viirs_snpp_jpss1_afd_gridstats_days')
print(f"Number of aspen fires: {grid.size().getInfo()}")
grid = grid.select(['grid_index', 'Fire_ID', 'Ig_Date', 'Last_Date', 'max_date', 'first_obs', 'last_obs'])
print(grid.first().propertyNames().getInfo())

Number of aspen fires: 875
['system:index', 'grid_index', 'Fire_ID', 'max_date', 'first_obs', 'Ig_Date', 'Last_Date', 'last_obs']


In [3]:
# get a list of unique active fire days
fire_days = grid.aggregate_array('first_obs').getInfo()
fire_days = set(fire_days)
print(f"Number of unique 'fire days': {len(fire_days)}")

# get the min and max DOY
fire_days_doy = [datetime.strptime(day, "%Y-%m-%d").timetuple().tm_yday for day in fire_days]
doy_min = min(fire_days_doy)
doy_max = max(fire_days_doy)
print(f"Fire DOY range: {doy_min} to {doy_max}")

Number of unique 'fire days': 453
Fire DOY range: 96 to 317


In [4]:
# Load the gridmet image collection
gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')
print(f"\ngridMET bands available for analysis:\n\n{gridmet.first().bandNames().getInfo()}\n")


gridMET bands available for analysis:

['pr', 'rmax', 'rmin', 'sph', 'srad', 'th', 'tmmn', 'tmmx', 'vs', 'erc', 'eto', 'bi', 'fm100', 'fm1000', 'etr', 'vpd']



In [5]:
# select our variables of interest:
# 1. Vapor Pressure Deficit and 2. Energy Release Component
gridmet = gridmet.select(['vpd','erc'])
print(gridmet.first().bandNames().getInfo())

['vpd', 'erc']


In [6]:
def process_fire_day(ftr):
    """ 
    Calculates gridmet statistics for daily grids (dissolved)
    Including the deviation from long-term average for time-of-year
    """

    ######################################################
    # calculate the long-term average for the time-of-year
    # get the fire days (ignition, cessation)
    ig_doy = ee.Date(ftr.get('Ig_Date')).advance(-14,'days').getRelative('day', 'year')
    ls_doy = ee.Date(ftr.get('Last_Date')).advance(14,'days').getRelative('day', 'year')

    # define the 'deviation' band names
    attrs = ee.List(gridmet.first().bandNames()) # grab the band names for renaming
    attrs = attrs.map(lambda atr: ee.String(atr).cat('_dv'))
    
    # average for time-of-year of fire
    lta = gridmet.filter(
        ee.Filter.calendarRange(ig_doy, ls_doy, 'day_of_year')
    ).mean().rename(attrs)      

    ##################################
    # get daily fire dates valid range
    first_date = ee.Date(ftr.get('first_obs')) # day of first observation
    end_date = ee.Date(ftr.get('first_obs')).advance(2, 'days') # advance a couple days
    
    # filter the gridmet to day of first observation
    gridmet_day = gridmet.filterDate(first_date, end_date).first()
    gridmet_anom = gridmet_day.subtract(lta).rename(attrs) # get the deviation
    image = gridmet_day.addBands(gridmet_anom)
    
    # run the reduction
    stats = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=ftr.geometry().bounds().buffer(100),
        scale=4000
    )

    return ee.Feature(None, stats).set({
        'Fire_ID': ftr.get('Fire_ID'),
        'first_obs': ftr.get('first_obs')
    })

# map the function across gridcells
results = ee.FeatureCollection(grid.map(process_fire_day))
results.limit(10)

In [7]:
sample = results.limit(10).getInfo()
props = [f['properties'] for f in sample['features']]
df = pd.DataFrame(props)
df = df[['Fire_ID', 'first_obs', 'erc', 'erc_dv', 'vpd', 'vpd_dv']]
df.head(10)

,Fire_ID,first_obs,erc,erc_dv,vpd,vpd_dv
0,33,2017-06-15,84.816327,17.413339,2.209796,0.677873
1,33,2017-06-16,86.164894,19.351884,2.241755,0.778100
2,33,2017-06-17,87.000000,20.710945,2.300000,0.911641
3,33,2017-06-18,84.000000,17.710945,1.970000,0.581641
4,70,2017-06-06,47.000000,-16.339294,1.320000,0.080339
5,70,2017-06-08,67.000000,3.660706,1.410000,0.170339
6,70,2017-06-09,70.000000,6.660706,1.530000,0.290339
7,70,2017-06-10,71.000000,7.660706,1.610000,0.370339
8,70,2017-06-11,72.000000,8.625309,1.420000,0.179758
9,70,2017-06-12,74.000000,10.316851,1.552857,0.307552


In [8]:
# Remove geometry
def remove_geometry(ftr):
    return ftr.setGeometry(None)
results = results.map(remove_geometry)
results = results.select(['Fire_ID','first_obs','erc','vpd','erc_dv','vpd_dv'])

# Export the table. 
export_task = ee.batch.Export.table.toDrive(
    collection=results,
    description='gridstats_gridmet',
    fileNamePrefix='gridstats_gridmet',
    fileFormat='CSV', 
    folder='GRIDMET'
)

export_task.start() # Start the export task
print("Export to Earth Engine Asset started!")
# Monitor the task until it's finished
monitor_export(export_task, 120) 

Export to Earth Engine Asset started!
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting fo